In [413]:
import numpy as np
import time
from selenium import webdriver
from typing import List
import pandas as pd


In [414]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

chrome_options = Options()

In [415]:
BASE_URL = "https://www.legaseriea.it/it/serie-a/archivio"
chromedriver = "/home/alesssandro/sw/chromedriver"
driver = webdriver.Chrome (executable_path="/home/alesssandro/sw/chromedriver")
browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)
browser.get(BASE_URL)
wait = WebDriverWait(browser, 10)

<ipython-input-415-4b5c5c8358ec>:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)


In [416]:
home_team_path = "//div[@class='col-xs-6 risultatosx']/h4[@class='nomesquadra']"
away_team_path = "//div[@class='col-xs-6 risultatodx']/h4[@class='nomesquadra']"
date_path = "//div[@class='datipartita']/p/span"
city_path = "//div[@class='datipartita']/p"

matches_list = {'Date': [], 'Home Team': [], 'Away Team': [], 'City': []}


In [417]:
def changeCityValue():
    cleaned_list = [] #temporary list with cleaned values
    values = matches_list["City"]
    
    for line in values:
        temp = (line.split("(",1)[1]).split(")")[0] #extraction of text written in brackets in p tag 
        cleaned_list.append(temp)
    matches_list["City"].clear()
    
    for city in cleaned_list:
        matches_list["City"].append(city)

In [418]:
def getValues(path_name, dictionary_key):
 
    for value in browser.find_elements_by_xpath(path_name):
        matches_list["{}".format(dictionary_key)].append(value.text)

In [419]:
def getData():
    getValues(date_path,"Date")
    getValues(home_team_path,"Home Team")
    getValues(away_team_path,"Away Team")
    getValues(city_path,"City")


In [420]:
def matchDays(): #iteration of all season matches
    
    box_calendar_path = "//ul[1]/li[@class='box_Ngiornata_andata']"#first round box path
    matches_counter = len(browser.find_elements_by_xpath(box_calendar_path)*2) #count of season matchdays
    
    for element in range(1,(matches_counter)+1):
        current_season = browser.find_element_by_xpath("//select[@id='archivio_stagione_id']/option[@selected='selected']")#string of season got from dropdown menu
        wait.until(lambda brow: len(brow.find_elements_by_xpath(box_calendar_path)) == (matches_counter / 2))
        getData()
        browser.get("https://www.legaseriea.it/it/serie-a/archivio/{}/UNICO/UNI/{}".format(current_season.text,element));#url change to iterate each matchday

In [421]:
def seasonCicle(): #iteration of seasons 
    
    season_number = browser.find_elements_by_xpath("//select[@id='archivio_stagione_id']/option") #in case we want to iterate all archive season, use it in final range below
    browser.find_element_by_id("archivio_stagione_id").click();

    for element in range(1,21): #range until season 2000-2001
        browser.find_element_by_xpath("//option[{}]".format(element)).click();#season selection from dropdown menu
        matchDays()#iteration of matchdays for each season

In [422]:
seasonCicle()
changeCityValue()

In [423]:
df = pd.DataFrame(data = matches_list)
df.to_csv('./matches_list.csv', index=None)